# RNN 용 데이터 자르기

RNN 모델은 주로 연속적인 데이터를 처리할 때 많이 사용함. 주가, 환율, 금리, 국제유가 등등  
그러다보니 연속되는 데이터를 만났을때, 어떻게 데이터를 나눠야할지, 데이터 분리에 대한 어려움을 겪게 됨.  

## split 함수 만들기 (다:1)
1~10까지의 데이터로 시작해봄 

In [3]:
# 1. 데이터
import numpy as np 
dataset = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

이 데이터로 RNN에 쓰고, 5개의 연속된 데이터로 그 다음의 값을 예측하는 모델을 만들고자 함.  
1~ 4까지의 데이터를 가지고 5를 예측하고, 2 ~ 5까지의 데이터를 가지고 6을 예측하고, 마지막으로 6 ~ 9까지의 데이터로 10을 예측하는 데이터 셋으로 바꿔줌  
그러면 아래와 같은 데이터가 될 것임  

|x|y|
|:---:|:---:|
|1, 2, 3, 4|5|
|2, 3, 4, 5|6|
|3, 4, 5, 6|7|
|4, 5, 6, 7|8|
|5, 6, 7, 8|9|
|6, 7, 8, 9|10|

x 는 6행 4열이고, y 는 6행 1열임. 이렇게 값이 얼마 되지 않을때는 직접입력해도 충분하지만  
데이터가 매우 많을때는 불가능함 

In [5]:
def split_xy1(dataset, timestep):
    x, y = list(), list()
    for i in range(len(dataset)):
        end_number = i + timestep
        if end_number > len(dataset) - 1:
            break
        tmp_x, tmp_y = dataset[i:end_number], dataset[end_number]
        x.append(tmp_x)
        y.append(tmp_y)
    return np.array(x), np.array(y)

x, y = split_xy1(dataset, 4)
print(x, "\n", y)

[[1 2 3 4]
 [2 3 4 5]
 [3 4 5 6]
 [4 5 6 7]
 [5 6 7 8]
 [6 7 8 9]] 
 [ 5  6  7  8  9 10]


위 함수는 연속된 데이터를 자르기 위해 만든 함수  
RNN의 input_shape는 (samples, time_steps, feature) 로 들어가게됨. 데이터를 RNN 모델에 맞는 shape 로 만들어줘야 한다는 뜻  

dataset은 우리가 나누고자 하는 데이터셋 time_steps 는 몇개의 컬럼(열)로 자를건지를 나타냄. 

`x, y = list(), list()` : 우선 리턴해줄 x, y를 리스트로 정의해줌 

`for i in range(len(dataset)):` : dataset의 개수만큼 for 문을 돌림

`end_number = i + time_steps` : 마지막 숫자가 몇인지를 정의함. 첫번째 반복문에서는 i=0 이므로 0+4=4, 마지막 숫자는 4가 됨.  

`if end_number > len(dataset) -1` : 방금 구한 마지막 숫자가 dataset의 전체길이 -1 보다 크다면 반복문을 나오게 됨. (end_number가 10이면 종료됨)  
   `break`

`tmp_x, tmp_y = dataset[i:end_number], dataset[end_number]` : 총 time_step 수 만큼 tmp_x 에 저장하고, end_number의 요소를 tmp_y 변수에 저장한 다음, 리스트에 전달해줌  
`x.append(tmp_x)`  
`y.append(tmp_y)`

In [6]:
x, y = split_xy1(dataset, 3)
print(x, "\n", y)

[[1 2 3]
 [2 3 4]
 [3 4 5]
 [4 5 6]
 [5 6 7]
 [6 7 8]
 [7 8 9]] 
 [ 4  5  6  7  8  9 10]


## split 함수 만들기2 (다:다)
데이터셋을 time_steps 만큼 잘라서 x값을 만들고, y 값 역시 원하는 만큼 잘라서 만드는 함수를 만들어봄.  
RNN모델을 구현할때, 주식으로 비유하자면 3일전,2일전,1일전의 주식 데이터를 가지고, 내일과 모레의 주가를 예측하는 모델을 만드는 것이 가능함.  
3개의 time_steps 로 x 를 만들고, 2개의 y컬럼을 만드는 함수를 만들어 봄

In [7]:
# 1. 데이터
import numpy as np 
dataset = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [12]:
def split_xy1(dataset, timestep, y_column):
    x, y = list(), list()
    for i in range(len(dataset)):
        x_end_number = i + timestep
        y_end_number = x_end_number + y_column
        
        if y_end_number > len(dataset):
            break
            
        tmp_x, tmp_y = dataset[i: x_end_number], dataset[x_end_number : y_end_number]
        x.append(tmp_x)
        y.append(tmp_y)
        
    return np.array(x), np.array(y)

x, y = split_xy1(dataset, 4, 2)
print(x, "\n", y)
print('x shape : ',x.shape)
print('y shape : ',y.shape)

[[1 2 3 4]
 [2 3 4 5]
 [3 4 5 6]
 [4 5 6 7]
 [5 6 7 8]] 
 [[ 5  6]
 [ 6  7]
 [ 7  8]
 [ 8  9]
 [ 9 10]]
x shape :  (5, 4)
y shape :  (5, 2)
